In [9]:
import numpy as np
import pandas as pd
from kernels import * 
from learning_models import *
from tools import *
import pandas as pd
import numpy as np
from time import time 
from sklearn.svm import SVC
from tqdm import tqdm
from autoreload import superreload
import operator

In [27]:
def spectrum_kernel(X, length):
    all_sequence = {}
    
    for idx in range(len(X)):
        data = X[idx]
        for i in range(len(data)-length + 1):
            seq1 = data[i:i+length]
            if seq1 in all_sequence:
                if idx in all_sequence[seq1]:
                    all_sequence[seq1][idx] += 1
                else:
                    all_sequence[seq1][idx] = 1
            else:
                all_sequence[seq1] = {}
                all_sequence[seq1][idx] = 1
    
    kernel = np.zeros((len(X), len(X)))
    
    for seq in all_sequence:
        for key1 in all_sequence[seq]:
            for key2 in all_sequence[seq]:
                kernel[key1][key2] += all_sequence[seq][key1]*all_sequence[seq][key2]
                
    return kernel

In [28]:
def embedding_mismatch_kernel(X, lengths, mismatch, verbose = False):
    
    all_sequences_index = {}
    id_last_seq = 0
    for data in X:
        for i in range(len(data)-lengths + 1):
            seq = data[i:i+lengths]
            if seq not in all_sequences_index:
                all_sequences_index[seq] = id_last_seq
                id_last_seq += 1

    
    vectors = np.zeros((len(X),len(all_sequences_index)))
    
    for idx in tqdm(range(len(X))):
        data = X[idx]
        for i in range(len(data)-lengths + 1):
            seq = data[i:i+lengths]
            vectors[idx, all_sequences_index[seq]] += 1
            if mismatch >= 1:
                for seq_mis in one_mismatch_away(seq, True):
                    if seq_mis in all_sequences_index:
                        vectors[idx, all_sequences_index[seq_mis]] += 1/2
            if mismatch >= 2:
                for seq_mis in two_mismatch_away(seq):
                    if seq_mis in all_sequences_index:
                        vectors[idx, all_sequences_index[seq_mis]] += 1/3
     
    if verbose:
        print(all_sequences_index)
        print('Embedding :')
        print(vectors)
    return np.dot(vectors,vectors.T)

In [29]:
def one_mismatch_away(s, codon = False):
    res = []
    if codon :
        all_letters = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'Z']
    else :
        all_letters = ['A','T','C','G']
    for i,letter in enumerate(s):
        if letter not in all_letters:
            print('Please use only letters in {}'.format(all_letters))
        for l in all_letters:
            if letter != l:
                res.append(str(s[:i]) + l + str(s[i+1:]))
    return res

In [30]:
def two_mismatch_away(s):
    res = []
    for a in one_mismatch_away(s):
        for t in one_mismatch_away(a):
            if t not in res and t != s and t not in one_mismatch_away(s):
                res.append(t)
    return res

In [6]:
def submitResults(filename, y_pred_final):
    '''
    Creates, from a 1d array of final predicted values, a file named filename formated appropriately in order to make a
    submission on the Kaggle platform.
    '''
    y = np.concatenate([y_pred_final[i] for i in [0,1,2]])
    with open("data/submission/{}.csv".format(filename), 'w') as f:
        string = "Id,Bound\n"
        for j in range(0,3000):
            string += str(j)+','+str(int(y[j]))+'\n'
        f.write(string)
    print("----------------- Prediction written on {}.csv ---------------".format(filename))

In [7]:
X = ['TTCG',
    'ATCG',
    'ATCG',
    'ATCA']
m1 = embedding_mismatch_kernel(X,3,1, False)
print(m1)

100%|██████████| 4/4 [00:00<00:00, 6505.32it/s]

[[ 2.5   2.25  2.25  2.  ]
 [ 2.25  2.5   2.5   2.25]
 [ 2.25  2.5   2.5   2.25]
 [ 2.    2.25  2.25  2.5 ]]


In [13]:
def translate_dna(cds):

    codontable = {
    'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
    'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
    'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
    'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
    'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
    'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
    'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
    'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
    'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
    'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
    'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
    'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
    'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
    'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
    'TAC':'Y', 'TAT':'Y', 'TAA':'Z', 'TAG':'Z',
    'TGC':'C', 'TGT':'C', 'TGA':'Z', 'TGG':'W',
    }
    proteinsequence = ''
    
    if len(cds)%3 != 0:
        print('Séquence pas divisible par 3')
        return proteinsequence
    for n in range(0,len(cds),3):
        if cds[n:n+3] in codontable:
            proteinsequence += codontable[cds[n:n+3]]
        else:
            raise ValueError('Pas dans le code génétique')
    return proteinsequence

In [15]:
file = 0
X_train = pd.read_csv("data/Xtr{}.csv".format(file), sep=' ',header=None)[0].values.tolist()
Y_train = pd.read_csv("./data/Ytr{}.csv".format(file), index_col=0)['Bound'].values

X_trans = [translate_dna(X[:99]) for X in X_train]

In [19]:
%%time
X_trans = [translate_dna(X[:99]) for X in X_train]

CPU times: user 54.8 ms, sys: 1.58 ms, total: 56.3 ms
Wall time: 55.7 ms


## Cross validation to launch

In [42]:
y_pred_final = []
length = [3,8,8]
nb_iter = 3
lambdas = [np.logspace(-2, 1, 7, endpoint=True),
           np.logspace(-2, 1, 7, endpoint=True),
           np.logspace(-2, 1, 7, endpoint=True)]

for file in [0,1,2]:
    X_train = pd.read_csv("data/Xtr{}.csv".format(file), sep=' ',header=None)[0].values.tolist()
    Y_train = pd.read_csv("./data/Ytr{}.csv".format(file), index_col=0)['Bound'].values
    print("----- File {} read ------".format(file))

    
    for align in [0,1,2]:
        X_trans = [translate_dna(X[align:align+99]) for X in X_train]

        res = {}

        for lmdb in lambdas[file]:
            res[lmdb] = [0,0]

        begin = time()
        K_train = embedding_mismatch_kernel(X_trans,length[file],1)
        end = time()
        print("----- Kernel {} computed in {} sec ------".format(file, end - begin))


        for i in tqdm(range(nb_iter)):
            for lmdb in lambdas[file]:
                X_train_c, X_test_c, Y_train_c, Y_test_c, K_train_c, K_test_c = K_train_test_split(X_trans,Y_train,K_train,test_size=0.2)
                Y_train_c.shape = (Y_train_c.shape[0])
                Y_test_c.shape = (Y_test_c.shape[0])
                Y_train_c.shape = (Y_train_c.shape[0])
                model = SVM(lmbd=lmdb)
                model.train(K_train_c, Y_train_c)
                Y_test_pred = list(map(int,model.predict(K_test_c)))
                Y_train_pred = list(map(int,model.predict(K_train_c)))
                res[lmdb][1] += accuracy_score(Y_train_c, Y_train_pred)/nb_iter
                res[lmdb][0] += accuracy_score(Y_test_c, Y_test_pred)/nb_iter

        print("Best for file {}, align : {}".format(file, align))
        sorted_x = sorted(res.items(), key=operator.itemgetter(1))

        print(sorted_x[-1])

  2%|▏         | 30/2000 [00:00<00:06, 296.36it/s]

----- File 0 read ------


  0%|          | 0/3 [00:00<?, ?it/s]

----- Kernel 0 computed in 6.532015085220337 sec ------


  2%|▏         | 35/2000 [00:00<00:05, 343.98it/s]

...

Best for file 1, align : 0
(0.031622776601683791, [0.53916666666666668, 1.0])


  0%|          | 0/3 [00:00<?, ?it/s]

----- Kernel 1 computed in 17.07411813735962 sec ------


 33%|███▎      | 1/3 [00:21<00:43, 21.95s/it]

KeyboardInterrupt: 

## Submission

In [458]:
from sklearn.svm import SVC

y_pred_final = []
length = [11,11,8]
l = [0.1668, 0.0220, 1.3226]

for file in [0,1,2]:
    X_train = pd.read_csv("data/Xtr{}.csv".format(file), sep=' ',header=None)[0].values.tolist()
    Y_train = pd.read_csv("./data/Ytr{}.csv".format(file), index_col=0)['Bound'].values
    X_test = pd.read_csv("data/Xte{}.csv".format(file), sep=' ',header=None)[0].values.tolist()

    big_X = np.concatenate((X_train, X_test),axis=0)
    
    print("----- File {} read ------".format(file))

    res = []
    if kernels_computed:
        
    begin = time()
    big_K = embedding_mismatch_kernel(big_X,length[file],2)
    end = time()
    print("----- Kernel {} computed in {} sec ------".format(file, end - begin))

    K_train = big_K[:len(X_train),:len(X_train)]
    K_test = big_K[len(X_train):,:len(X_train)]
    
    clf = SVM(lmbd=l[file], loss='squared_hinge')
    clf.train(K_train, Y_train)
    y_pred = clf.predict(K_test)
    
    y_pred_final.append(y_pred)

    print("----- File {} predicted ------".format(file))


submitResults("SixthSubmission", y_pred_final)


----- File 0 read ------
----- Kernel 0 computed in 3.0994415283203125e-06 sec ------
----- File 0 predicted ------
----- File 1 read ------
----- Kernel 1 computed in 2.1457672119140625e-06 sec ------
----- File 1 predicted ------
----- File 2 read ------
----- Kernel 2 computed in 2.1457672119140625e-06 sec ------
----- File 2 predicted ------
----------------- Prediction written on SixthSubmission.csv ---------------


Travail sur le dataset 0:
Mismatch 6-0 : lambda = 0.02712 score = 0.7575
Mismatch 9-1 : lambda = 0.007742 score = 0.7692
Mismatch 11-2 : lambda = 0.1668 score = 0.7713


Travail sur le dataset 1:
Mismatch 6-0 : lambda = 0.05994 score = 0.8825
Mismatch 9-1 : lambda = 0.01584 score = 0.9050
Mismatch 11-2 : lambda = 0.0220 score = 0.9075

Travail sur le dataset 2:
Mismatch 5-0 : lambda = 0.00774 score = 0.64225
Mismatch 7-1 : lambda = 0.46415 score = 0.6652
Mismatch 8-2 : lambda = 1.3226 score = 0.67175

In [50]:
a = np.array([1,2])

In [51]:
a[2]

IndexError: index 2 is out of bounds for axis 0 with size 2